In [1]:
# 指令跟随格式转换

import json
import os

def convert_to_chat_format(input_file_path, output_file_path):
    """
    将原始数据文件转换为对话格式 (Chat Format) 的 JSON Lines 文件。

    Args:
        input_file_path (str): 输入文件的路径 (每行一个JSON对象).
        output_file_path (str): 输出文件的路径.
    """
    # 确保输出目录存在
    output_dir = os.path.dirname(output_file_path)
    if output_dir:
        os.makedirs(output_dir, exist_ok=True)

    processed_lines = 0
    try:
        # 使用 'with' 语句确保文件被正确关闭
        with open(input_file_path, 'r', encoding='utf-8') as infile, \
             open(output_file_path, 'w', encoding='utf-8') as outfile:
            
            for line in infile:
                # 移除行尾可能存在的空白符并处理空行
                line = line.strip()
                if not line:
                    continue

                try:
                    # 解析原始的 JSON 数据
                    original_data = json.loads(line)

                    # 提取需要的数据
                    user_content = original_data.get("original_user_request")
                    assistant_content = original_data.get("optimized_response")

                    # 确保关键字段存在，否则跳过该行
                    if not user_content or not assistant_content:
                        print(f"警告：跳过缺少 'original_user_request' 或 'optimized_response' 的行: {line}")
                        continue

                    # 构建新的对话格式字典
                    chat_data = {
                        "messages": [
                            {
                                "role": "user",
                                "content": user_content
                            },
                            {
                                "role": "assistant",
                                "content": assistant_content
                            }
                        ]
                    }

                    # 将新的字典转换为 JSON 字符串并写入输出文件
                    # ensure_ascii=False 确保中文字符被正确写入，而不是被转义
                    outfile.write(json.dumps(chat_data, ensure_ascii=False) + '\n')
                    processed_lines += 1

                except json.JSONDecodeError:
                    print(f"警告：跳过格式错误的 JSON 行: {line}")
                except KeyError as e:
                    print(f"警告：跳过缺少关键字段 {e} 的行: {line}")

        print(f"\n处理完成！")
        print(f"总共处理了 {processed_lines} 行。")
        print(f"输出文件已保存至: {output_file_path}")

    except FileNotFoundError:
        print(f"错误：输入文件未找到: {input_file_path}")
    except Exception as e:
        print(f"发生未知错误: {e}")

if __name__ == "__main__":
    input_file = "../dataset/optimized_jds.jsonl"
    output_file = "../dataset/ai-cars-instruction.jsonl"
    convert_to_chat_format(input_file, output_file)


处理完成！
总共处理了 1299 行。
输出文件已保存至: ../dataset/ai-cars-instruction.jsonl
